## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
cd /content/drive/MyDrive/Colab Notebooks/TextMining_WordEmbedding

/content/drive/MyDrive/Colab Notebooks/TextMining_WordEmbedding


In [9]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Rea

In [10]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [11]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [12]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [13]:
corpus[:50]

['애니메이션에서나 볼 수 있었던 자동차 변신 로봇이 일본에서 실제로 만들어진다',
 "11일 소프트뱅크 계열 로봇 제조업체인 아스라테크(Asratec)와 산세이 테크놀로지, 브레이브 로보틱스가 3사 합작으로 변신 로봇 '제이 다이트 라이드(J-deite RIDE)'를 제작한다고 발표했다",
 '이 로봇은 아스라테크와 브레이브 로보틱스가 참여한 로봇 제조 계획 "프로젝트 제이 다이트\'의 일환으로 지난해 6월부터 개발이 진행되고 있다',
 "하드웨어 설계 ·제작은 브레이브 로보틱스가 맡고 제어용 소프트웨어 '브이시도(V-Sido)'의 개발은 아스라테크가 담당한다",
 '산세이 테크놀로지는 로봇의 양산화를 위해서 합류했다',
 "'제이 다이트 라이드'는 내년에 완성돼 테마 파크와 각종 이벤트 참여, 기업 홍보 등의 목적으로 쓰이게 된다",
 "이후에는 기체의 크기를 키워 2020년에는 5m에 달하는 '제이 다이트 오리지널'로 재탄생할 예정이다",
 "디자인은 애니메이션 '기동전사 건담', '달려라 번개호' 등 유명 애니메이션에 등장하는 로봇을 창조한 오오카와라 쿠니오가 맡았다",
 "오오카와라는 '제이 다이트 라이드' 외에 프로젝트에서 제작할 예정인 다른 로봇의 디자인도 맡는다",
 "'제이다이트 라이드'의 머리 부분 스케치",
 '언뜻 보기에도 건담이 연상된다',
 '이 로봇은 10초만에 자동차로 변신할 수 있으며 전체 길이는 약 4m이다',
 '로봇 모드에서는 두 다리로 걸을 수 있다',
 '차량 모드에서는 평범한 자동차와 마찬가지로 사람이 운전석에 앉아 조작하면 된다',
 "'철인28호'처럼 로봇과 떨어져 원격 조종을 해도 된다",
 '최대 2명이 탈 수 있다',
 '로봇이 두 다리로 걷는 속도는 시속 5km에 불과하지만 발 부분의 바퀴를 이용하면 한시간에 최고 30km를 갈 수 있다',
 '차로 변신했을 때 최고 속도는 시속 60km이다',
 "한편 '제이다이트 라이드'의 1.5m짜리 시제품은 14일(현지시간) 미국 올랜도에서 열리는 '국제 놀이기구 엑스

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]

In [104]:
okt = Okt()

In [105]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.
def CustomTokenization(sentence):
  tokenizing = []
  stop = ['Punctuation','Josa',]
  
  for i, j in okt.pos(sentence, stem = True, norm= True):
      if j in stop:
           continue
      tokenizing.append(i)
  return tokenizing

In [106]:
CustomTokenization('최대 2명이 탈 수 있다 ㅋㅋㅋㅋㅋㅋㅋ',)

['최대', '2', '명', '탈', '수', '있다', 'ㅋㅋㅋ']

In [107]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

# 2번

이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]

In [122]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(size = 200, min_count = 0, alpha = 0.03, )

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 4123


In [123]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 30 )

(486344, 546210)

In [124]:
model_no_pretrained.wv.vocab.keys()

dict_keys(['애니메이션', '볼', '수', '있다', '자동차', '변신', '로봇', '일본', '실제', '만들어지다', '11일', '소프트뱅크', '계열', '제조업체', '아스', '테크', 'Asratec', '오다', '산세이', '테크놀로지', '브레이브', '로보틱스', '3', '사', '합작', '제이', '다이트', '라이드', 'J', 'deite', 'RIDE', '를', '제작', '다', '고', '발표', '하다', '이', '참여', '제조', '계획', '프로젝트', '의', '일환', '지난해', '6월', '부터', '개발', '진행', '되다', '하드웨어', '설계', '맡다', '제어', '용', '소프트웨어', '브이', '시도', 'V', 'Sido', '담당', '양산', '화', '위해', '합류', '늘다', '내년', '완성', '돼다', '테마', '파크', '각종', '이벤트', '기업', '홍보', '등', '목적', '쓰이다', '이후', '기체', '크기', '키우다', '2020년', '에는', '5', 'm', '달', '오', '리지', '널', '로', '재탄생', '예정', '디자인', '기동', '전사', '건담', '달다', '번개', '호', '유명', '등장', '창조', '오카', '쿠니', '오가다', '외', '다른', '제이다', '트', '머리', '부분', '스케치', '언뜻', '보기', '연상', '10초', '만에', '전체', '길이', '약', '4', '모드', '두', '다리', '걸', '차량', '평범하다', '마찬가지', '사람', '운전', '석', '앉다', '조작', '철인', '28', '처럼', '떨어지다', '원격', '조종', '해도', '최대', '2', '명', '탈', '걷다', '속도', '시속', 'km', '불과하다', '발', '바퀴', '이용', '한', '시간', '최고', '30', '갈다', '차로', '때', 

In [125]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_no_pretrained.wv.most_similar('배터리')

[('밀도', 0.999550461769104),
 ('리튬이온', 0.9995116591453552),
 ('슈퍼', 0.999485433101654),
 ('자기장', 0.9993532299995422),
 ('변형', 0.9993163347244263),
 ('남', 0.9993143677711487),
 ('배다', 0.9993118643760681),
 ('만나다', 0.9993094205856323),
 ('저렴하다', 0.9993009567260742),
 ('나아가다', 0.9993000030517578)]

In [126]:
model_no_pretrained.wv.most_similar('학부')

[('서울대', 0.9992984533309937),
 ('신소재공학', 0.998945951461792),
 ('소정', 0.9987667202949524),
 ('네덜란드', 0.9987517595291138),
 ('박', 0.9986966252326965),
 ('성균관대', 0.9985861778259277),
 ('강', 0.9983707666397095),
 ('KAIST', 0.9983146786689758),
 ('수색', 0.9982097744941711),
 ('하이파', 0.9981451034545898)]

In [127]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
model_pretrained = Word2Vec(size = pretrained.wv.vector_size, min_count = 0)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200
현재 vocabulary size(V) : 4123


In [128]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 31727


In [130]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs =30 )

(538674, 546210)

In [131]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_pretrained.wv.most_similar('학부')

[('대학원', 0.7320373058319092),
 ('대학', 0.7151874303817749),
 ('학과', 0.6896836757659912),
 ('예과', 0.6891124248504639),
 ('공학부', 0.678754448890686),
 ('전문학교', 0.6712114810943604),
 ('약학', 0.6517139673233032),
 ('부교수', 0.6490755677223206),
 ('조교수', 0.6464223861694336),
 ('문학부', 0.6449837684631348)]

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

[여러분의 설명]

In [ ]:
#성능 차이가 많이 나는 것 같습니다. 아마 학습할 수 있는 vocabulary size의 크기에 따른 변화가 아니였나 싶습니다.

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]

In [132]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

4123

In [133]:
# embedding 결과 가져오기
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_no_pretrained.wv[i])

In [137]:
pca = PCA(random_state=0).fit_transform(vector)
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 500).fit_transform(vector)

In [138]:
print(tsne.shape)
tsne

(4123, 2)


array([[ 2.9509822e-05,  1.3662411e-05],
       [ 4.3701479e-05,  5.1163857e-05],
       [ 1.6337615e-05, -5.4634682e-05],
       ...,
       [-3.9805494e-05, -3.0705643e-05],
       [ 6.7067871e-05, -3.9802864e-05],
       [ 4.0578456e-05,  2.1048383e-05]], dtype=float32)

In [141]:
# 시각화해보세요!
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = unique_text,
    mode='markers',
    marker=dict(
        size=5,
        
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()


/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning: plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin

  DeprecationWarning,


In [142]:
data = go.Scatter(
    x=pca[:,0],
    y=pca[:,1],
    text = unique_text,
    mode='markers',
    marker=dict(
        size=5,
        
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

In [144]:
# t-SNE
iter_v = [500, 1000]
perplexity_v = [5, 20, 35,  50]
for i in iter_v:
  for j in perplexity_v:
    tsne = TSNE(n_components = 2, random_state = 0, perplexity = 40, learning_rate = 0.05, init = 'random', n_iter = 500).fit_transform(vector)

    data = go.Scatter(
      x=tsne[:,0],
      y=tsne[:,1],
      text = unique_text,
      mode='markers',
      marker=dict(
        size=5,
        #color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
    )

    layout = go.Layout(
      autosize=False,
      width=1000,
      height=1000,
      margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
      ),
    )

    fig = go.Figure(data=[data], layout=layout)
    fig.show()


# 시각화



/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 수고하셨습니다!